In [1]:
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pylab
import utils
import os

In [2]:
# Load metadata and features.
tracks = utils.load('data/fma_metadata/tracks.csv')
genres = utils.load('data/fma_metadata/genres.csv')

rockGenreIds = []
hiphopGenreIds = []
popGenreIds = []
jazzGenreIds = []
elecGenreIds = []
classicalGenreIds = []
for key in tracks['track','genres'].keys():
    if len(tracks['track','genres'][key]) >= 1:
        if tracks['track','genres'][key][0] == 12:
            rockGenreIds.append(key)
        if tracks['track','genres'][key][0] == 21:
            hiphopGenreIds.append(key)
        if tracks['track','genres'][key][0] == 10:
            popGenreIds.append(key)
        if tracks['track','genres'][key][0] == 4:
            jazzGenreIds.append(key)
        if tracks['track','genres'][key][0] == 15:
            elecGenreIds.append(key)
        if tracks['track','genres'][key][0] == 5:
            classicalGenreIds.append(key)

('track', 'tags')
('album', 'tags')
('artist', 'tags')
('track', 'genres')
('track', 'genres_all')


In [3]:
def get_filenames(audioDirectory, ids):
    file_list = []
    for count, Id in enumerate(ids):
        try:
            Id = ids[count]
            #print count
            Id = str(Id)
            while(len(Id) < 6):
                Id = "0" + Id
            #print (rockId)
            path = os.path.join(audioDirectory, Id[0:3])
            files = os.listdir(path)
            audioFilename = os.path.join(path, Id + ".mp3")
            if os.path.isfile(audioFilename):
                file_list.append(audioFilename)
        except:
            print count, Id
        
    return file_list

audioDirectory = "/ubc/cs/research/tracking-raid/candice/project/fma_medium"
rock_song_list = get_filenames(audioDirectory, rockGenreIds)
hiphop_song_list = get_filenames(audioDirectory, hiphopGenreIds)
elec_song_list = get_filenames(audioDirectory, elecGenreIds)
classical_song_list = get_filenames(audioDirectory, classicalGenreIds)

In [ ]:
filename = "/ubc/cs/research/tracking-raid/candice/project/DreamingInMusic/data/small/000/000998.mp3"
print os.path.isfile(filename)

In [4]:
print len(rock_song_list)
print len(hiphop_song_list)
elec_song_list = elec_song_list[:2000]
print len(elec_song_list)
print len(classical_song_list)
song_list = rock_song_list + hiphop_song_list + elec_song_list + classical_song_list
print len(song_list)

2406
1957
2000
448
6811


In [5]:
for i in range(10):
    print song_list[i]

/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000247.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000249.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000255.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000564.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000605.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000608.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000633.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000635.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000690.mp3
/ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000720.mp3


In [ ]:
# Find the min and max of spectrogram in all audio in directory
duration = 9.98 # duration of audio

minAmp = float("inf")
maxAmp = -float("inf")
for audioFilename in song_list:
    for start_time in [0, 10, 20]:
        try:
            x, sr = librosa.load(audioFilename, sr=None, mono=True, duration=duration, offset=start_time)
        except:
            print audioFilename
            continue
        if len(x) > 0:
            #Convert audio to a complex valued spectrogram
            spectro = librosa.core.stft(x)

            #Separate out amplitude and phase from complex valued spectrogram
            mag, phase = librosa.core.magphase(spectro)
            #print ("mag", mag)
            #print ("phase",phase)

            #Get the decibal version from power spectrogram
            #This is the value that should be stored for training
            powerToDb = librosa.power_to_db(mag, ref=np.max)
            locMin = np.amin(powerToDb)
            locMax = np.amax(powerToDb)
            #print locMin, locMax
            minAmp = min(minAmp, locMin)
            maxAmp = max(maxAmp, locMax)
            
print ("minAmp", minAmp)
print ("maxAmp", maxAmp)

In [20]:
import scipy.misc

minAmp = -80.0
maxAmp = 1.9073486e-06
def scaleSpectro(x, new_size):
    #print ("before scaling", x)
    x = (x - minAmp)/(maxAmp - minAmp)
    #print ("after scaling", x)
    y = scipy.misc.imresize(x, new_size, mode='L', interp='nearest')
    return y
def unscaleSpectro(x, new_size, minAmp, maxAmp):
    x = scipy.misc.imresize(x, new_size, mode='L', interp='nearest')
    x = x/255.0
    x = minAmp + x*(maxAmp - minAmp)
    return x
def convert_file_to_spectro(audio_filename):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    try:
        x1, sr1 = librosa.load(audio_filename, sr=None, mono=True, duration=9.98, offset=0)
    except:
        return
    try:
            x2, sr2 = librosa.load(audio_filename, sr=None, mono=True, duration=9.98, offset=10)
    except:
        return
    try:
        x3, sr3 = librosa.load(audio_filename, sr=None, mono=True, duration=9.98, offset=20)
    except:
        return
    #Convert audio to a complex valued spectrogram
    spectro1 = librosa.core.stft(x1)
    spectro2 = librosa.core.stft(x2)
    spectro3 = librosa.core.stft(x3)
    if spectro1 is not None and spectro2 is not None and spectro3 is not None:
        #Separate out amplitude and phase from complex valued spectrogram
        mag1, phase1 = librosa.core.magphase(spectro1)
        mag2, phase2 = librosa.core.magphase(spectro2)
        mag3, phase3 = librosa.core.magphase(spectro3)

        #Get the decibal version from power spectrogram
        #This is the value that should be stored for training
        powerToDb1 = librosa.power_to_db(mag1, ref=np.max)
        powerToDb2 = librosa.power_to_db(mag2, ref=np.max)
        powerToDb3 = librosa.power_to_db(mag3, ref=np.max)
        return powerToDb1, powerToDb2, powerToDb3
    else:
        return None, None, None

In [21]:
spectro_1, spectro_2, spectro_3 = convert_file_to_spectro(song_list[0])
#spectro_2 = convert_file_to_spectro(song_list[1])
print spectro_1
print spectro_2
print spectro_3

[[-32.10647583 -35.17770386 -35.13761902 ..., -20.74552917 -22.76325798
  -12.42108345]
 [-32.5177536  -27.79302597 -27.45069695 ..., -20.14678001 -17.63718796
  -12.72651672]
 [-33.8721199  -27.12277222 -21.54623795 ..., -21.62534142 -17.47250557
  -14.71877098]
 ..., 
 [-67.71305084 -61.27999115 -62.68729401 ..., -61.99525452 -55.33665466
  -47.77518463]
 [-73.31834412 -60.96147919 -56.9090271  ..., -60.05062866 -55.87080383
  -48.63137817]
 [-77.46194458 -60.91259003 -55.89610291 ..., -56.63808441 -59.34059143
  -49.69758606]]
[[-19.07538986 -20.0002594  -22.55150604 ..., -24.92151642 -21.60593987
  -12.51545811]
 [-21.77766609 -17.39510918 -18.46725845 ..., -18.72729301 -18.92308807
  -13.1489706 ]
 [-17.18309402 -16.38022804 -17.05885887 ..., -17.69587708 -18.12091064
  -12.17409611]
 ..., 
 [-44.79163742 -47.81441116 -62.04322433 ..., -60.63745117 -55.75390244
  -48.10538864]
 [-44.60562134 -47.72201157 -60.77893066 ..., -58.64273834 -62.50084686
  -48.45567322]
 [-44.44965363 -4

In [30]:
def save_spectros(song_list, genre_name, save_dir):
    num_songs = len(song_list)
    num_val_songs = num_songs / 5
    for i, song in enumerate(song_list):
        if i < num_val_songs:
            save_ddir = os.path.join(save_dir, 'val')
        else:
            save_ddir = os.path.join(save_dir, 'train')
        if not os.path.exists(save_ddir):
            os.makedirs(save_ddir)
        save_subdir = os.path.join(save_ddir, genre_name)
        if not os.path.exists(save_subdir):
            os.makedirs(save_subdir)
        if convert_file_to_spectro(song) is not None:
            spectro1, spectro2, spectro3 = convert_file_to_spectro(song)
        else:
            return
        if spectro1 is not None and spectro2 is not None and spectro3 is not None:
            scaled_spectro1 = scaleSpectro(spectro1, (512, 512))
            scaled_spectro2 = scaleSpectro(spectro2, (512, 512))
            scaled_spectro3 = scaleSpectro(spectro3, (512, 512))
            save_name1 = os.path.join(save_subdir, str(3*i+1) + '.jpg')
            scipy.misc.imsave(save_name1, scaled_spectro1)
            save_name2 = os.path.join(save_subdir, str(3*i+2) + '.jpg')
            scipy.misc.imsave(save_name2, scaled_spectro2)
            save_name3 = os.path.join(save_subdir, str(3*i+3) + '.jpg')
            scipy.misc.imsave(save_name3, scaled_spectro3)

In [29]:
save_dir = '/ubc/cs/research/tracking-raid/candice/project/dataset/mag_512'
save_spectros(rock_song_list, 'rock', save_dir)
save_spectros(hiphop_song_list, 'hiphop', save_dir)
save_spectros(classical_song_list, 'classical', save_dir)
save_spectros(elec_song_list, 'elec', save_dir)